In [60]:
%run "../common-WadeCopy1.ipynb"

In [203]:
import pickle
clfs = pickle.load( open( "trainedclassifications.p", "rb" ) )
numtimesinvalid = pickle.load( open( "numtimesinvalid.p", "rb" ) )
#print (clfs['Random Forest'])
fileName = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q=WITH%20rows%20AS%20(SELECT%20*,ROW_NUMBER()%20OVER%20(ORDER%20BY%20mobile_id,%20id%20DESC)%20AS%20rn%20FROM%20loc%20WHERE%20(record_type%20%3C%3E%20%27active%27%20OR%20record_type%20IS%20NULL))%20SELECT%20*%20FROM%20rows%20loc1%20WHERE%20(loc1.rn,loc1.mobile_id)%20%20IN%20((SELECT%20loc2.rn,%20loc2.mobile_id%20FROM%20rows%20loc2%20WHERE%20loc2.predictedvalidity=-1)%20UNION%20(SELECT%20loc2.rn-1,%20loc2.mobile_id%20FROM%20rows%20loc2%20WHERE%20loc2.predictedvalidity=-1)%20UNION%20(SELECT%20loc2.rn%2B1,%20loc2.mobile_id%20FROM%20rows%20loc2%20WHERE%20loc2.predictedvalidity=-1)%20)%20%20LIMIT%201000&type=html"
dftopredictstart = LoadDataSet(fileName, columns=None, checkForDateTime=False);



In [204]:
from datetime import datetime,timedelta


columns = '''
id, 
stored_at, measured_at, is_valid, api_key, version, record_type, session_num, 
mobile_id, user_id, caller_ip, 
lat, lon, accuracy, speed, bearing, alt, device_temp, device_pressure, device_humidity, device_light, medication, 
weather_time, temperature_min, temperature_max, humidity, no2, pressure, wind, clouds_sky, notes, attr, the_geom, 
location, activity, battery_level, rn"
'''
toDrop= [k.strip() for k in ''' api_key, version,  session_num, 
mobile_id, 
user_id, 
caller_ip, device_temp, device_pressure, device_humidity, device_light, medication, weather_time, 
temperature_min, temperature_max, humidity, no2, pressure, wind, clouds_sky, notes, attr, 
the_geom,weatherchecked,pef,fev, record_type, battery_level, location, activity,markedvalidity,

'''.split(",")]
dftopredict = dftopredictstart.drop(toDrop, axis=1, errors="ignore")
#print(dftopredict)


In [205]:
usedpredictions = []
stored_at = []
measured_at = []
is_valid = []
#record_type = []
lat = []
lon = []
accuracy = []
speed = []
bearing = []
alt = []
#location = []
#activity = []
#battery_level = []
rn = []

previousinvalids = []
secfromprev = []
sectonext = []
distfromprev = []
disttonext = []
spdfromprev = []
spdtonext = []
angle = []



for k in range(dftopredict['predictedvalidity'].value_counts()[-1]):
    unpredictedidx = dftopredict['predictedvalidity'][dftopredict['predictedvalidity']==-1].index[k]
    if unpredictedidx==0 or unpredictedidx==len(dftopredict.index)-1:
        continue;
    if dftopredict['rn'][unpredictedidx]-1 != dftopredict['rn'][unpredictedidx-1] or dftopredict['rn'][unpredictedidx]+1 != dftopredict['rn'][unpredictedidx+1]:    
        continue;
    
    usedpredictions.append(dftopredict['id'][unpredictedidx])
    stored_at.append(dftopredict['stored_at'][unpredictedidx])
    measured_at.append(dftopredict['measured_at'][unpredictedidx])
    is_valid.append(dftopredict['is_valid'][unpredictedidx])
    #record_type.append(dfLinvalid['record_type'][invalididx])
    lat.append(dftopredict['lat'][unpredictedidx])
    lon.append(dftopredict['lon'][unpredictedidx])
    accuracy.append(dftopredict['accuracy'][unpredictedidx])
    speed.append(dftopredict['speed'][unpredictedidx])
    bearing.append(dftopredict['bearing'][unpredictedidx])
    alt.append(dftopredict['alt'][unpredictedidx])
    #location.append(dfLinvalid['location'][unpredictedidx])
    #activity.append(dfLinvalid['activity'][unpredictedidx])
    #battery_level.append(dfLinvalid['battery_level'][unpredictedidx])
    rn.append(dftopredict['rn'][unpredictedidx])

    curdatetime = datetime.strptime(dftopredict['measured_at'][unpredictedidx].split('.')[0],'%Y-%m-%d %H:%M:%S')
    prevdatetime = datetime.strptime(dftopredict['measured_at'][unpredictedidx+1].split('.')[0],'%Y-%m-%d %H:%M:%S')
    nextdatetime = datetime.strptime(dftopredict['measured_at'][unpredictedidx-1].split('.')[0],'%Y-%m-%d %H:%M:%S')
    
    #seconds from previous
    secfromprev.append((curdatetime - prevdatetime).seconds)

    #seconds to next
    sectonext.append((nextdatetime-curdatetime).seconds)


    curlat = dftopredict['lat'][unpredictedidx]

    curlon = dftopredict['lon'][unpredictedidx]


    key = str(curlat) + ', ' + str(curlon)
    if key in numtimesinvalid:
        previousinvalids.append(numtimesinvalid[key])
    else:
        previousinvalids.append(0)
        
    nextlat = dftopredict['lat'][unpredictedidx-1]
    nextlon = dftopredict['lon'][unpredictedidx-1]
        
    prevlat = dftopredict['lat'][unpredictedidx+1]
    prevlon = dftopredict['lon'][unpredictedidx+1]
        
    #miles to next
    disttonext.append(math.sqrt(69.1*(curlat-nextlat)*69.1*(curlat-nextlat)+(69.1*(curlon - nextlon)*cos(nextlat/57.3))*(69.1*(curlon - nextlon)*cos(nextlat/57.3))))
    #print(disttonext[k])
        
    #miles from prev
    distfromprev.append(math.sqrt(69.1*(curlat-prevlat)*69.1*(curlat-prevlat)+(69.1*(curlon - prevlon)*cos(prevlat/57.3))*(69.1*(curlon - prevlon)*cos(prevlat/57.3))))
    #print(distfromprev[k])
        
        
        
    #Speed in MPH to next
    if sectonext[len(sectonext)-1] != 0:
        spdtonext.append(disttonext[len(disttonext)-1]/(sectonext[len(sectonext)-1]/3600))
    else:
        spdtonext.append(None)
    #print(spdtonext[k])
        
    #Speed in MPH from prev
    if secfromprev[len(secfromprev)-1] != 0:
        spdfromprev.append(distfromprev[len(distfromprev)-1]/(secfromprev[len(secfromprev)-1]/3600))
    else:
        spdfromprev.append(None)
    #print(spdfromprev[k])
        
    #Angle in degrees between previous point, next point, and current point
    if distfromprev[len(distfromprev)-1]*disttonext[len(disttonext)-1] != 0:
        distancebetweennextandprev = math.sqrt(69.1*(prevlat-nextlat)*69.1*(prevlat-nextlat)+(69.1*(prevlon - nextlon)*cos(nextlat/57.3))*(69.1*(prevlon - nextlon)*cos(nextlat/57.3)))
        lawofcos = (math.pow(distancebetweennextandprev,2) - math.pow(distfromprev[len(distfromprev)-1],2) - math.pow(disttonext[len(disttonext)-1],2))/(-2*distfromprev[len(distfromprev)-1]*disttonext[len(disttonext)-1])
        if lawofcos < -1:
            lawofcos = -1
        if lawofcos > 1:
            lawofcos = 1
        angle.append((180/math.pi)*math.acos(lawofcos))
    else:
        angle.append(None)
    #print(angle[k])
    
    
newdict = {'stored_at': stored_at,
'measured_at': measured_at,
'is_valid': is_valid,
#'record_type': record_type,
#'lat': lat,
#'lon': lon,
'accuracy': accuracy,
'speed': speed,
'bearing': bearing,
'alt': alt,
#'location': location,
#'activity': activity,
#'battery_level': battery_level,
'rn': rn,
'secfromprev': secfromprev,
'sectonext': sectonext,
'distfromprev': distfromprev,
'disttonext': disttonext,
'spdfromprev': spdfromprev,
'spdtonext': spdtonext,
'angle': angle,
'previousinvalids': previousinvalids}

dftopredictnew = pd.DataFrame(newdict)


dftopredictnew = dftopredictnew.drop('rn',axis=1)
dftopredictnew = dftopredictnew.drop('is_valid',axis=1)
#print(dftopredictnew)
dftopredictnew = prepareDF(dftopredictnew, makeCopy=True)
#print(dftopredictnew)




Dropping in prepareDF -  Index(['measured_at', 'stored_at'], dtype='object')


In [206]:
import urllib.request
import urllib.parse
prediction = clfs['Random Forest'].predict(dftopredictnew)

values=""
for k in range(len(prediction)):
    ididx = k+1;
    entryid = usedpredictions[k]
    predictedvalidity = prediction[k]+1
    values+="("+str(entryid)+","+str(predictedvalidity)+")"
    if k%10 != 0 and k != len(prediction)-1:
        values+=","
    if k%10 == 0 or k == len(prediction)-1:
        url = "UPDATE loc SET predictedvalidity = c.predictedvalidity FROM (values"+values+") as c(id, predictedvalidity) WHERE c.id = loc.id"
        url = "http://www.smartconnectedhealth.org/aura/webroot/db.jsp?q="+urllib.parse.quote_plus(url)+"&type=html"
        #print(url)
        response = urllib.request.urlopen(url)
        html = response.read()
        #print(html)
        values=""

print("Done predicting " + str(len(usedpredictions)) + " entries.")
print("Valid: "+str(sum(prediction==0))+" Invalid: "+str(sum(prediction==1)))
#response = urllib.request.urlopen(url)
#html = response.read()
#print(html)

Done predicting 538 entries.
Valid: 468 Invalid: 70
